# Simulation of Quantum Chromodynamics and Nuclear Astrophysics

Core-collapse supernovae are among the most catostrophic events in the universe, and play an important role in cosmological history, providing an engine by which elements heavier than iron can be synthesized. Although supernovae are ultimately driven by the microscopic mechanisms in nuclear physics, their simulation naively appears to be a problem of classical physics of transport and fluid dynamics. However, these supernovae generate a tremendous number of neutrinos, with the bulk of the energy of the supernova being carried away by the neutrinos.

For a complete treatment of the physics of a supernova, a critical question to answer is the exact
momentum and flavor distribution of the neutrinos during the course of the supernova. At such
large densities, traditional classical computational methods of kinetic theory become impractical, as
it is necessary to determine the time-evolution of the full many-body density matrix, and recently,
much effort in the field has been devoted to using quantum computation and quantum information
science as a way forward.

Tracing out a static nuclear and electron background that the neutrinos are embedded in, the hamiltonian for our system is the following:
$$\hat{H} = \hat{H}_1 + \hat{H}_2 $$
$$\hat{H}_1 = \sum_{f, f'}\sum_{s, s'} \int d^3 \vec{p}B_{ff',ss'}(t, \vec{p})\hat{a}_{f,s}^\dag(\vec{p})\hat{a}_{f',s'\vec{p}}$$

However, a lower bound to the computational gate complexity of our hamiltonian can be provided by considering a simplified case of the neutrino scattering hamiltonian known as the forward scattering model. As the forward scattering model is a strict subset of the full problem, any gate count and qubit requirements for the forward scattering problem lower bounds the difficulty of the full problem.

In the integrals over all possible momenta, we consider two decisions:
1. $\vec{p_1} = -\vec{p_3}$ and $\vec{p_2} = -\vec{p_1}$
2. $\vec{p_1} = -\vec{p_4}$ and $\vec{p_2} = -\vec{p_3}$
which represents the forward/back-scattering scenarios.

This model comes with several more restrictions:
- Neglect anti-neutrinos
- Restrict to a two-flavor model
- Fix the number of momentum nodes to be the number of neutrinos, N=K

We can then write the hamiltonian as an all-to-all Heisenberg model, yielding
$$\hat{H} = \sum_{i=1}^{N}\vec{b_i} \cdot \hat{\vec{\sigma_{i}}} + \frac{G_{F}\mu}{\sqrt{2}N} p_i \cdot p_j \hat{\vec{\sigma_i}} \cdot \hat{\vec{\sigma_{j}}} $$

Where now N is the number of neutrinos or equivalently momentum nodes, and $\vec{b}_i$ is the one-body coupling matrix, $\vec{p}_i$ is the momentum of the i-th neutrino, $\mu$ is the density of neutrinos, which we can take to be 1 with a redition of time after setting the one-body terms to zero, and $\hat{\vec{\sigma_i}}$ is a Pauli matrix operating on the flavor space of the i-th momentum mode. 

With an all-to-all connected architecture, the simplest Trotterization of this Hamiltonian yields a circuit with N depth for a single trotter step.

In [1]:
import cirq

# Initial State Preparation
A reasonable initial state is a product state, which for each neutrino is Gaussian distributed over momenta, with the momentum peaked at the nuclear scales at which the neutrinos are generated, i.e, 1 - 10 MeV. Product states suffice due to the uncorrelated production of the neutrinos, and the gaussian distribution localizes the neutrinos to specific regions of the supernova.

For determining a good initial state that provides information over how the thermalization proceeds as function of the inititla total spin along the z-direction, there are five cases we can consider. Despite their being more exotic states that we can consider, such states are irrelevant for this task. These five possible initial states are:
1. 50% qubits up state and the rest down
2. 60% qubits up state and the rest down
3. 70% qubits up state and the rest down
4. 80% qubits up state and the rest down
5. 90% qubits up state and the rest down

As such, we can clearly see that the cost for preparing the initial state is a constant, depth 1 operation. 

In [12]:
def initial_state_prep(n_qubits:int, spin_up_percent:float) -> cirq.Circuit:
    qubits_spin_up = int(spin_up_percent*n_qubits)
    qubits = [cirq.LineQubit(idx) for idx in range(qubits_spin_up)]
    spin_up_portion = [cirq.X(qb) for qb in qubits]

    circuit = cirq.Circuit()
    circuit.append(spin_up_portion)
    return circuit

initial_state_prep(10, 0.5)

0: ───X───

1: ───X───

2: ───X───

3: ───X───

4: ───X───

# Dynamics

The simplest case that can prove to be of scientific use will be to consider a gas of momentum plane-wave neutrinos, undergoing two-body interactions without a complicating matter background, in the relativistic limit. Then the one-body hamiltonian reduces reduces to the identity, being the conserved total kinetic energy of the system. Thus, the goal will be to see how equilibration is reached for flavor distribution of a single momentum mode for a given distribution of initial neutrino momenta and flavors.